In [74]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/grasp-and-lift-eeg-detection/train.zip
/kaggle/input/grasp-and-lift-eeg-detection/sample_submission.csv.zip
/kaggle/input/grasp-and-lift-eeg-detection/test.zip


In [75]:
import zipfile
with zipfile.ZipFile("../input/grasp-and-lift-eeg-detection/test.zip","r") as z:
    z.extractall(".")
with zipfile.ZipFile("../input/grasp-and-lift-eeg-detection/train.zip","r") as z:
    z.extractall(".")

In [76]:
import numpy as np
from zipfile import ZipFile
import matplotlib.pyplot as plt
import os
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import random
import torch.nn.functional as F
import argparse
from sklearn import metrics
from tqdm.notebook import tqdm
import gc
import shutil 

from sklearn.model_selection import train_test_split
import numpy as np
import pywt
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression
from glob import glob
import scipy
from scipy.signal import butter, lfilter, convolve, boxcar
from scipy.signal import freqz
from scipy.fftpack import fft, ifft
import os

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler,Normalizer,MinMaxScaler

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM, BatchNormalization, Conv2D, Flatten, MaxPooling2D, Dropout, Conv1D, LeakyReLU, MaxPool1D
from keras.optimizers import Adam
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,roc_curve,auc
import tensorflow as tf

In [77]:
%%time
FILE_PATH = '../input/grasp-and-lift-eeg-detection'
list_dir = os.listdir(FILE_PATH)

for zipfile in list_dir:
    with ZipFile(os.path.join(FILE_PATH, zipfile), 'r') as z:
        z.extractall()

CPU times: user 20.4 s, sys: 4.62 s, total: 25.1 s
Wall time: 27.1 s


In [78]:
#from something
def wavelet_denoising(x, wavelet='db2', level=1):
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/0.6745) * madev(coeff[-level])

    uthresh = sigma * np.sqrt(2*np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])

    return pywt.waverec(coeff, wavelet, mode='per')
def madev(d, axis=None):
    """ Mean absolute deviation of a signal """
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

In [79]:
sample_data = pd.read_csv('/kaggle/working/test/subj4_series9_data.csv')
sample_data.head()

,id,Fp1,Fp2,F7,F3,Fz,F4,F8,FC5,FC1,...,P7,P3,Pz,P4,P8,PO9,O1,Oz,O2,PO10
0,subj4_series9_0,-1,136,878,-519,-21,62,-259,-58,-6,...,400,265,151,77,163,65,140,19,-96,-102
1,subj4_series9_1,-56,99,838,-532,-10,180,-253,-61,-1,...,393,255,164,88,148,30,140,21,-111,-105
2,subj4_series9_2,-104,50,800,-474,-14,148,-239,-11,-2,...,366,236,148,82,141,35,113,20,-98,-77
3,subj4_series9_3,-70,48,869,-456,6,118,-228,29,-7,...,361,215,146,81,143,6,106,20,-110,-74
4,subj4_series9_4,-19,31,876,-413,25,119,-200,46,-2,...,373,222,135,99,164,43,120,48,-43,-39


In [80]:
from sklearn.preprocessing import StandardScaler,Normalizer,MinMaxScaler

scaler= StandardScaler()
def trainprep(filename):
    data = pd.read_csv(filename)
    events_fname = fname.replace('_data','_events')
    labels= pd.read_csv(events_fname)
    clean=data.drop(['id' ], axis=1)#remove id
    labels=labels.drop(['id' ], axis=1)#remove id
    return  clean,labels

def testprep(filename):
    data = pd.read_csv(filename)
    return data

def train_preprocess(X):
    X_prep=scaler.fit_transform(X)
    return X_prep
def test_preprocess(X):
    X_prep=scaler.transform(X)
    return X_prep

In [81]:

subjects = range(1,6)
from glob import glob
import pandas as pd
ids_tot = []
pred_tot = []
X_train_butter = []
from sklearn.model_selection import train_test_split
import numpy as  np

y_raw= []
raw = []
y_rawt= []
rawt = []
for subject in subjects:  
    filenames =  sorted(glob('train/subj%d_series*_data.csv' % (subject)))
    for filename in filenames:
      data,labels=trainprep(fname)
      raw.append(data)
      y_raw.append(labels)
    for filename in filenames:
      with open(fname) as myfile:
        head = [next(myfile) for x in range(10)]
        
X = pd.concat(raw)
y = pd.concat(y_raw)
X_train =np.asarray(X.astype(float))
y_train = np.asarray(y.astype(float))

#fs = 500.0
#owcut = 7.0
#highcut = 30.0


x_train_butter=wavelet_denoising(X_train)
x_train=train_preprocess(x_train_butter)
splitrate=-x_train.shape[0]//5*2
xval=x_train[splitrate:splitrate//2]
yval=y_train[splitrate:splitrate//2]
xtest=x_train[splitrate//2:]
ytest=y_train[splitrate//2:]
xtrain=x_train[:splitrate]
ytrain=y_train[:splitrate]

In [82]:
import numpy as np
load = 1
time_steps = 1000
subsample = 50
model = Sequential()
model.add(Conv2D(filters = 64, kernel_size = (7,7), padding = "same", activation = "relu", input_shape = (time_steps//subsample, 32, 1)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (5,5), padding = "same", activation = "relu", input_shape = (time_steps//subsample, 32, 1)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = "same", activation = "relu", input_shape = (time_steps//subsample, 32, 1)))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(32, activation = "relu"))
model.add(BatchNormalization())
model.add(Dense(6, activation = "sigmoid"))

adam = Adam(lr = 0.0001)

model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics = ['accuracy','mse'])

model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_36 (Conv2D)          (None, 20, 32, 64)        3200      
                                                                 
 batch_normalization_53 (Ba  (None, 20, 32, 64)        256       
 tchNormalization)                                               
                                                                 
 conv2d_37 (Conv2D)          (None, 20, 32, 64)        102464    
                                                                 
 batch_normalization_54 (Ba  (None, 20, 32, 64)        256       
 tchNormalization)                                               
                                                                 
 conv2d_38 (Conv2D)          (None, 20, 32, 64)        36928     
                                                                 
 batch_normalization_55 (Ba  (None, 20, 32, 64)      

In [83]:
def valgenerator():
    while 1:
        batch_size=32
        x_time_data = np.zeros((batch_size, time_steps//subsample, 32))
        yy = []
        for i in range(batch_size):
            random_index = np.random.randint(0, len(xval)-time_steps)
            x_time_data[i] = xval[random_index:random_index+time_steps:subsample]
            yy.append(yval[random_index + time_steps])
        yy = np.asarray(yy)
        yield x_time_data.reshape((x_time_data.shape[0],x_time_data.shape[1], x_time_data.shape[2],1)), yy
    

In [84]:
import time
start=time.time()
def generator(batch_size):
    while 1:
        x_time_data = np.zeros((batch_size, time_steps//subsample, 32))
        yy = []
        for i in range(batch_size):
            random_index = np.random.randint(0, len(xtrain)-time_steps)
            x_time_data[i] = xtrain[random_index:random_index+time_steps:subsample]
            yy.append(ytrain[random_index + time_steps])
        yy = np.asarray(yy)
        yield x_time_data.reshape((x_time_data.shape[0],x_time_data.shape[1],  x_time_data.shape[2],1)), yy
        
#history =model.fit_generator(generator(32), steps_per_epoch = 600, epochs = 50,validation_data=valgenerator(),
                              #validation_steps=200)
history =model.fit(generator(32), steps_per_epoch = 600, epochs = 50,validation_data=valgenerator(),
                              validation_steps=200)



Epoch 1/50
600/600 [==============================] - 9s 8ms/step - loss: 0.3914 - accuracy: 0.2535 - mse: 0.2237 - val_loss: 0.3743 - val_accuracy: 0.4133 - val_mse: 0.2504
Epoch 2/50
600/600 [==============================] - 5s 8ms/step - loss: 0.3708 - accuracy: 0.5265 - mse: 0.2076 - val_loss: 0.3301 - val_accuracy: 0.7716 - val_mse: 0.1553
Epoch 3/50
600/600 [==============================] - 5s 8ms/step - loss: 0.3362 - accuracy: 0.7966 - mse: 0.1656 - val_loss: 0.3158 - val_accuracy: 0.8223 - val_mse: 0.1472
Epoch 4/50
600/600 [==============================] - 5s 9ms/step - loss: 0.3255 - accuracy: 0.8317 - mse: 0.1472 - val_loss: 0.3557 - val_accuracy: 0.8477 - val_mse: 0.1071
Epoch 5/50
600/600 [==============================] - 5s 8ms/step - loss: 0.3199 - accuracy: 0.8434 - mse: 0.1394 - val_loss: 0.2846 - val_accuracy: 0.8547 - val_mse: 0.1109
Epoch 6/50
600/600 [==============================] - 5s 8ms/step - loss: 0.3084 - accuracy: 0.8440 - mse: 0.1332 - val_loss: 0.30